In [1]:
import os
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

In [2]:
fake = pd.read_csv('./news/Fake.csv')
real = pd.read_csv('./news/True.csv')
fake['target'] = "0"
real['target'] = "1"
fake.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [3]:
fake.shape

(23481, 5)

In [4]:
news = pd.concat([real, fake], axis = 0, join="inner", ignore_index=True)
news.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [5]:
import warnings
warnings.filterwarnings("ignore")
stop_words = set(stopwords.words('english'))
def filter_text(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    text1 = re.sub(r'[^a-zA-Z]', ' ', text)
    text2 = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text1, flags=re.MULTILINE)
    words = text2.lower().split()
    words = [w for w in words if not w in stop_words] 
    return ' '.join(words)

In [6]:
filter_text(news['text'][1])

'washington reuters transgender people allowed first time enlist u military starting monday ordered federal courts pentagon said friday president donald trump administration decided appeal rulings blocked transgender ban two federal appeals courts one washington one virginia last week rejected administration request put hold orders lower court judges requiring military begin accepting transgender recruits jan justice department official said administration challenge rulings department defense announced releasing independent study issues coming weeks rather litigate interim appeal occurs administration decided wait dod study continue defend president lawful authority district court meantime official said speaking condition anonymity september pentagon said created panel senior officials study implement directive trump prohibit transgender individuals serving defense department feb submit plan trump lawyers representing currently serving transgender service members aspiring recruits said

In [7]:
#(1)Clean data
news['clean_text'] = news.text.apply(filter_text)
news.head()

,title,text,subject,date,target,clean_text
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,washington reuters head conservative republica...
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,washington reuters transgender people allowed ...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,washington reuters special counsel investigati...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,washington reuters trump campaign adviser geor...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,seattle washington reuters president donald tr...


In [8]:
from  sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(news.clean_text, news.target, test_size=0.2, random_state=0)

In [9]:
f1 = open("train_contents.txt", "w")
f1.write('\n'.join(X_train))
f1.close()
f1 = open("train_labels.txt", "w")
f1.write('\n'.join(Y_train))
f1.close()
f1 = open("test_contents.txt", "w")
f1.write('\n'.join(X_test))
f1.close()
f1 = open("test_labels.txt", "w")
f1.write('\n'.join(Y_test))
f1.close()

In [10]:
train_texts = open('train_contents.txt').read().split('\n')
train_labels = open('train_labels.txt').read().split('\n')
test_texts = open('test_contents.txt').read().split('\n')
test_labels = open('test_labels.txt').read().split('\n')
all_texts = train_texts + test_texts
all_labels = train_labels + test_labels

In [11]:
import keras
VECTOR_DIR = 'GoogleNews-vectors-negative300.bin'

Using TensorFlow backend.


In [12]:
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2

In [13]:
#(2) tokenize and sequencing data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(all_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 115210 unique tokens.
Shape of data tensor: (44898, 500)
Shape of label tensor: (44898, 2)


In [14]:
#(3) split data set
p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]
print ('Train docs: '+str(len(x_train)))
print( 'Val docs: '+str(len(x_val)))
print ('Test docs: '+str(len(x_test)))

Train docs: 26938
Val docs: 8980
Test docs: 8980


In [15]:
#(4) load word2vec as embedding
import gensim
from keras.utils import plot_model
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(VECTOR_DIR, binary=True)
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
not_in_model = 0
in_model = 0
for word, i in word_index.items(): 
    if word in w2v_model:
        in_model += 1
        embedding_matrix[i] = np.asarray(w2v_model[word], dtype='float32')
    else:
        not_in_model += 1
print(str(not_in_model)+' words not in w2v model')
from keras.layers import Embedding
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

62564 words not in w2v model


In [16]:
#(5) training model
from tensorflow.python.framework import tensor_util
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding
from keras.models import Sequential

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()
plot_model(model, to_file='model.png',show_shapes=True)
exit(0)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print(model.metrics_names)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
model.save('word_vector_lstm.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          34563300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 602       
Total params: 35,285,102
Trainable params: 721,802
Non-trainable params: 34,563,300
_________________________________________________________________
['loss', 'acc']

Train on 26938 samples, validate on 8980 samples
Epoch 1/2
26938/26938 [==============================] - 1124s 42ms/step - loss: 0.2642

In [17]:
# testing model
print(model.evaluate(x_test, y_test))

8980/8980 [==============================] - 181s 20ms/step
[0.2183678489717715, 0.9111358523368835]
